# Assistant API with code interpreter
Code interpreter for Azure OpenAI is an azure hosted code execution environment.

In [ ]:
# Import relevant functionality
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables (set OPENAI_API_KEY and OPENAI_API_BASE in .env)
load_dotenv()
    
client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("AZURE_OPENAI_VERSION"),
    azure_endpoint = os.getenv("OPENAI_API_ENDPOINT")
    )

In [ ]:
# Create an assistant
assistant = client.beta.assistants.create(
    name="Data Visualization",
    instructions=f"You are a helpful AI assistant who makes interesting visualizations based on data." 
    f"You have access to a sandboxed environment for writing and testing code."
    f"When you are asked to create a visualization you should follow these steps:"
    f"1. Write the code."
    f"2. Anytime you write new code display a preview of the code to show your work."
    f"3. Run the code to confirm that it runs."
    f"4. If the code is successful display the visualization."
    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
    tools=[{"type": "code_interpreter"}],
    temperature=0.5,
    model="gpt-4-1106-preview" #You must replace this value with the deployment name for your model.
)

In [ ]:
print(assistant.model_dump_json(indent=2))

{
  "id": "asst_gPqGMFUE3dxIztMnR72hhxTI",
  "created_at": 1717514168,
  "description": null,
  "instructions": "You are a helpful AI assistant who makes interesting visualizations based on data.You have access to a sandboxed environment for writing and testing code.When you are asked to create a visualization you should follow these steps:1. Write the code.2. Anytime you write new code display a preview of the code to show your work.3. Run the code to confirm that it runs.4. If the code is successful display the visualization.5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
  "metadata": {},
  "model": "gpt-4-1106-preview",
  "name": "Data Visualization",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ],
  "response_format": "auto",
  "temperature": 0.5,
  "tool_resources": {
    "code_interpreter": {
      "file_ids": []
    },
    "file_search": null
  },
  

In [ ]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_lXNomvISDb1PDEJBcyFOAXyX', created_at=1717514168, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [ ]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Illustrate a sinecurve with matplotlib"
)

thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

import time
from IPython.display import clear_output

start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)

messages = client.beta.threads.messages.list(
  thread_id=thread.id
) 

print(f'Status: {status}')
print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
print(messages.model_dump_json(indent=2))

Status: completed
Elapsed time: 0 minutes 26 seconds
{
  "data": [
    {
      "id": "msg_ra1HyVJvNTy4CEW0IKohIQP5",
      "assistant_id": "asst_gPqGMFUE3dxIztMnR72hhxTI",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "image_file": {
            "file_id": "assistant-QEFlpbWFF6TlbsCBYofD7yh4",
            "detail": null
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here is the visualization of a sine curve using Matplotlib. The curve represents the sine function across a range of x values from 0 to \\(2\\pi\\)."
          },
          "type": "text"
        }
      ],
      "created_at": 1717514191,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_DRNVNGfmugFgdKN3joiAFFH2",
      "status": null,
      "thread_id": "thread_lXNomvISDb1P